In [1]:
import keras
import tensorflow as tf
from keras import backend as K
import numpy as np

num_cores=4
num_CPU=1
num_GPU=0
config=tf.ConfigProto(intra_op_parallelism_threads=num_cores,
                     inter_op_parallelism_threads=num_cores,
                     allow_soft_placement=True,
                     gpu_options=tf.GPUOptions(allow_growth=True),
                     device_count={'CPU':num_CPU,'GPU':num_GPU})
session = tf.Session(config=config)
K.set_session(session)

from keras.models import Sequential,model_from_json
from keras.layers import Dense,Dropout,Activation
from keras.optimizers import SGD,RMSprop,Adamax,Adam,Adagrad
from keras.losses import mean_absolute_percentage_error
from keras.utils import np_utils,generic_utils
from keras import metrics
import error
import math
import multiprocessing as mp
import json
import sys

/home/shy/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Parametres
====

In [2]:
iteration=4
error_bound=0.05
net_A=[2,4,4,1]
net_C=[2,4,2]

epochA=2
epochC=5
batch_size=256

error_type="absolute_error"

def error_compare(error_type):
    if (error_type=="absolute_error"):
        return error.absolute_error
    else:
        return error.relative_error

In [3]:
error_measure=error_compare(error_type)

Input data processing
======

In [4]:
def format_data(data):
    try:
        if (len(data.shape)==1):
            return data.reshape((data.shape[0],1))
        else:
            return data
    except:
        print("Error! data is not a numpy object,format_data failed!")
        exit(0)

def load_data(app_name):
    x_train=np.loadtxt('../data/'+app_name+'/train.x')
    y_train=np.loadtxt('../data/'+app_name+'/train.y')
    x_test=np.loadtxt('../data/'+app_name+'/test.x')
    y_test=np.loadtxt('../data/'+app_name+'/test.y')
    return format_data(x_train),format_data(y_train),format_data(x_test),format_data(y_test)

Metrics on outputs
======

In [5]:
def gen_accept(error_bound):
    def accept(v0,v1):
        return error_measure([v0],[v1])<=error_bound
    return accept

Accelerator & Classifier
===

In [6]:
def AcceleratorModel(net_list,type=0):
    if (len(net_list)<2):
        print('Error! input accelerator net structure is wrong!')
        exit(0)
    model=Sequential()
    model.add(Dense(net_list[1],input_shape=(net_list[0],)))
    model.add(Activation('tanh'))
    
    for i in net_list[2:]:
        model.add(Dense(i))
        model.add(Activation('tanh'))
        
    prop=[RMSprop(0.01),RMSprop(0.008),RMSprop(0.006)]
    #prop=[Adagrad(),Adagrad(),Adagrad()]
    model.compile(loss='mse',optimizer=prop[type],metrics=[metrics.mse]) #不同application训练一定要不同loss function吗？待查
    return model

In [7]:
def ClassifierModel(net_list):
    if (len(net_list)<2):
        print('Error! input classifier net structure is wrong!')
        exit(0)
    model=Sequential()
    model.add(Dense((net_list[1]),input_shape=(net_list[0],)))
    if (len(net_list)>2):
        model.add(Activation('tanh'))
        for i in net_list[2:-1]:
            model.add(Dense(net_list[i]))
            model.add(Activation('tanh'))
        model.add(Dense(net_list[-1]))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',optimizer=RMSprop(0.01))
    return model

Training Block
====

In [12]:
def main(app_name,iteration,error_bound):
    #Load data
    x_train_origin,y_train_origin,x_test,y_test=load_data(app_name) #numpy type
    print ("Training data shape:",x_train_origin.shape,y_train_origin.shape)#Print the input shape, compare with the net_A and net_C, 
    print('Testing data shape:',x_test.shape,y_test.shape)                  #check whether they match each other
    
    #Setting about approximator and classifier(There may be more than one Approximator, refer to acceleratorModel())
    A=[AcceleratorModel(net_A,i)for i in range(3)]
    C=ClassifierModel(net_C)
    print("The Model A:")
    A[0].summary()
    print("The Model C:")
    C.summary()
    
    
    #每一个iteration：
    #1.用上一轮的数据训练A（Using all data in train set in the 1st iteration）
    #2.用所有Input做输入，predict A的output（这样能保证之前没有训练到的数据有加入训练数据的机会）
    #3.A的输出和标准output比较，得出error，进而得出整个train数据集每一个数据能不能用A跑的label，用这个label训练C。
    #4.让C对整个trainset进行预测，得到predictC。结合predictA和predictC产生下一个iteration的用于训练A的data（*算法）
    #5.evaluate部分
    
    x_train=x_train_origin #第一轮训练A的数据
    y_train=y_train_origin
    for i in range(iteration):
        
        A[0].fit(x_train,y_train,epochs=epochA,batch_size=batch_size,verbose=1) #Train A
        
        predictA=A[0].predict(x_train_origin) #Predict for A on x_train
        
        labelA=np.array([error_measure([predictA[i]],[y_train_origin[i]])<=error_bound for i in range(len(predictA))])
        
        print labelA.shape
        print labelA
        
    
    
    

In [13]:
main('bessel_Jnu',1,0.05)

('Training data shape:', (70000, 2), (70000, 1))
('Testing data shape:', (30000, 2), (30000, 1))
The Model A:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 4)                 12        
_________________________________________________________________
activation_23 (Activation)   (None, 4)                 0         
_________________________________________________________________
dense_24 (Dense)             (None, 4)                 20        
_________________________________________________________________
activation_24 (Activation)   (None, 4)                 0         
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 5         
_________________________________________________________________
activation_25 (Activation)   (None, 1)                 0         
Total params: 37
Trainable param

NameError: global name 'numpy' is not defined